In [1]:
# HIDDEN
from plotting_utils import *
# import cbayes.distributions
import scipy.stats as ss
import ipywidgets as wd
import numpy as np
from matplotlib import pyplot as plt
import cbayes.sample 
from IPython.display import clear_output, display
import yaml

In [2]:
# HIDDEN
plt.rcParams['figure.figsize'] = 20,10
plt.rcParams['font.size'] = 32
# plt.style.use('dark_background')
plt.rcParams['lines.linewidth'] = 5
plt.rcParams['font.family'] = 'Georgia'
dirname, fname = 'prop', 'social'

In [3]:
# HIDDEN

N = int(5E3) 

D = {'donor_1': {'amount': {'dist': 'beta',
   'kwds': {'a': 1.0, 'b': 1.0, 'loc': 10000.0, 'scale': 40000.0}},
  'confidence': {'dist': 'beta',
   'kwds': {'a': 3.0, 'b': 3.0, 'loc': 0.0, 'scale': 1.0}}},
 'donor_2': {'amount': {'dist': 'beta',
   'kwds': {'a': 1.5, 'b': 3.0, 'loc': 10000.0, 'scale': 40000.0}},
  'confidence': {'dist': 'beta',
   'kwds': {'a': 6.0, 'b': 3.0, 'loc': 0.0, 'scale': 1.0}}},
 'donor_3': {'amount': {'dist': 'beta',
   'kwds': {'a': 7.0, 'b': 3.0, 'loc': 10000, 'scale': 2500000}},
  'confidence': {'dist': 'beta',
   'kwds': {'a': 3.0, 'b': 7.0, 'loc': 0, 'scale': 1}}}}

d = ['donor 1', 'donor 2', 'donor 3']
p = ['amount', 'confidence']
# A = paramdict(d,p)
# TAB = assumptions(D, preview=True)

In [ ]:
# with open('database.yaml', 'w') as db:
#     yaml.dump(D, db)
    
# with open('database.yaml', 'r') as db:
#     D = yaml.load(db)

In [4]:
# HIDDEN
def gifts(amount=10000,
            confidence=0.5,
            **kwds):
    return amount*confidence

def total(UNIT_DICT, func):
    # sum costs over all strains
    return np.sum([func(**UNIT_DICT[k]) for k in UNIT_DICT.keys()], axis=0)

# def show_gifts(TAB, max_percentile):
#     U_new = extract(TAB)
#     S = cbayes.sample.generate_sample_set_from_dict(U_new, int(N)) 
#     V = cbayes.sample.generate_sample_dict(S)
#     output = total(V,gifts) # proportion of day
#     plot_results(output, 
#                     max_percentile = max_percentile,
#                     show_range = False,
#                     scaling_x = 1E3,
#                     bins  = 1000,
#                     save=False)
#     return 



# Office of Advancement x Mind the Math
![logo](logo.png)

# The "Story"

We have three donors under consideration, with different levels of _risk_ and _reward_. How these values are determined are at present, not considered. Many models that leverage data can be used to improve the descriptions of uncertainty for these two variables.

In our model, we assign the (temporary) names `amount` and `confidence` to address the respective reward and associated risk, respectively. 
- `amount` is the _dollar amount of a gift at realization_. 
    - Financial models, life expectancy tables, and initial investment amount can be used to determine this.
- `confidence` is an assessment of how likely (probability between 0 and 1) the gift is to be realized.
    - This can be based on the best intuition of the people with whom a donor is in contact, or predictive analytics that use available datasets. 

## Our Donor List
- **Donor 1**: _The wildcard._ Very little information known about them. We assign a 50/50 chance of realization, only a lower/upper bound is communicated to the Office.
- **Donor 2**: _The dependable alumna._ Considerable donation with small potential for good growth, quite likely to realize gift. Same range as first donor, but more confidence in the amount to be realized due to short life expectancy. 
- **Donor 3**: _The hopeful._ Emotionally driven individual with an aggressively invested fund and long time-period. Potential for sizable donation but considerable chance of the gift being revoked.

#### Please Consider
This interface is re-purposed from other projects owned by Mind the Math. It is solely for purposes of demonstration. 
One immediate change will be that donors are shown in a list instead of tabulated at top. Sliders for confidence should probably be replaced/ammended with a number of preset buttons for ease-of-use. 
This interface is live and interactive (though running on limited computational resources), so feel free to experiment and change the "stories" of each of our donors, and please pardon any lag you may experience. 

# Interface

In [5]:
# HIDDEN
donor_list = D.keys()
dropdown_list = wd.Dropdown(options=donor_list)
min_donation = wd.IntText(value=1E4, description='', continuous_update=False)
max_donation = wd.IntText(value=5E4, description='', continuous_update=False)
pessimism_amount = wd.FloatSlider(value=1, min=0.25, max=10, step=0.25, continuous_update=False, description='')
optimism_amount = wd.FloatSlider(value=1, min=0.25, max=10, step=0.25, continuous_update=False, description='')
pessimism_confidence = wd.FloatSlider(value=1, min=0.25, max=10, step=0.25, continuous_update=False, description='')
optimism_confidence = wd.FloatSlider(value=1, min=0.25, max=10, step=0.25, continuous_update=False, description='')


label_list = [wd.Label('Minimum Donation'), wd.Label('Maximum Donation'), wd.Label('Optimism'), wd.Label('Pessimism')]

def get_vals(args):
    # scrapes the slider data and writes it to the database D
    
    b_amt, a_amt = pessimism_amount.value, optimism_amount.value
    b_con, a_con = pessimism_confidence.value, optimism_confidence.value
    
    loc = min_donation.value
    scale = max_donation.value - loc
    amount_distribution = ss.distributions.beta(a=a_amt, b=b_amt, scale=scale, loc=loc)
    confidence_distribution = ss.distributions.beta(a=a_con, b=b_con, scale=1, loc=0)
    return amount_distribution, confidence_distribution

# out = wd.Output()
# @out.capture()
def write_vals(args):
    idx = dropdown_list.value
    amount_distribution, confidence_distribution = get_vals(args)
    a_amt, b_amt = amount_distribution.kwds['a'], amount_distribution.kwds['b']
    loc, scale = amount_distribution.kwds['loc'], amount_distribution.kwds['scale']
    a_con, b_con = confidence_distribution.kwds['a'], confidence_distribution.kwds['b']
    with open('database.yaml', 'r') as db:
        D = yaml.load(db)
    D[idx]['amount']['kwds'] = {'loc': loc, 'scale': scale, 'a': a_amt, 'b': b_amt}
    D[idx]['confidence']['kwds'] = {'loc': 0, 'scale': 1, 'a': a_con, 'b': b_con}
    with open('database.yaml', 'w') as db:
        yaml.dump(D, db)
#     print('updated D', dropdown_list.value)
# display(out)

write_button = wd.Button(description='Save Donor Profile')
write_button.on_click(write_vals)


def read_vals(args):
    # get the data from database D and set the sliders to that.
    idx = dropdown_list.value
#     idx = args['new']
    with open('database.yaml', 'r') as db:
        D = yaml.load(db)
    loc = D[idx]['amount']['kwds']['loc']
    scale = D[idx]['amount']['kwds']['scale']
    b_amt = D[idx]['amount']['kwds']['b']
    a_amt = D[idx]['amount']['kwds']['a']
    b_con = D[idx]['confidence']['kwds']['b']
    a_con = D[idx]['confidence']['kwds']['a']
    
#     amount_distribution = ss.distributions.beta(a=a_amt, b=b_amt, scale=scale, loc=loc)
    # set all the sliders 
    pessimism_amount.value, optimism_amount.value = b_amt, a_amt
    pessimism_confidence.value, optimism_confidence.value = b_con, a_con
    min_donation.value = loc
    max_donation.value = scale + loc
    preview_dists(None)

plot_out = wd.Output()
f = plt.figure()
A = f.add_subplot(1,2,2)
C = f.add_subplot(1,2,1)
plt.close(f)
@plot_out.capture()
def preview_dists(args):
    amount_distribution, confidence_distribution = get_vals(args)
    loc, scale = amount_distribution.kwds['loc'], amount_distribution.kwds['scale']
    a_con, b_con = confidence_distribution.kwds['a'], confidence_distribution.kwds['b']
    a_amt, b_amt = amount_distribution.kwds['a'], amount_distribution.kwds['b']
    n = 100
    a_x = np.linspace(loc, loc+scale, n)
    c_x = np.linspace(0, 1, n)
    a_y = amount_distribution.pdf(a_x)
    c_y = confidence_distribution.pdf(c_x) 
    clear_output(wait=True)
    A.clear()
    C.clear()
    if a_amt > b_amt:
        A.plot(a_x, a_y, color='xkcd:blue grey', lw=5)
    else:
        A.plot(a_x, a_y, color='xkcd:dark red', lw=5)
        
    if a_con > b_con:
        C.plot(c_x, c_y, color='xkcd:dried blood', lw=5)
    else:
        C.plot(c_x, c_y, color='xkcd:chartreuse', lw=5)
    
    A.set_yticks([])
    C.set_yticks([])
    C.set_xticks([])
    
    x_labels = A.get_xticks()
    scaling_label = '(thousands)'
    scaling_x = 1000
    A.set_xticklabels(['%2.1f'%(x/scaling_x) for x in x_labels])
    A.set_xlabel("Amount\n%s"%scaling_label)
    C.set_xlabel("\nConfidence")
    C.set_xticks([0, 0.5, 1])
    C.set_xticklabels(['low', 'middle', 'high'])

    display(f)
    

results_out = wd.Output()
g = plt.figure()
O = g.add_subplot(1,1,1)
plt.close(g)
@results_out.capture()
def show_result(args):
    with open('database.yaml', 'r') as db:
        D = yaml.load(db)
    S = cbayes.sample.generate_sample_set_from_dict(D, int(N)) 
    V = cbayes.sample.generate_sample_dict(S)
    output = total(V,gifts) # proportion of day
    G = ss.gaussian_kde(output)
    scaling_x = 1000
    max_percentile = 100
    scaling_label = '(thousands)'
    n = 1000
    clear_output(wait=True)
    O.clear()
    o_x = np.linspace(np.min(output), np.max(output), n)
    o_y = G.pdf(o_x)
    
    
    
    O.plot(o_x, o_y, lw=5, color='xkcd:deep purple', ls='--')
#     O.plot(o_x, o_y, lw=5, color='xkcd:chartreuse')
    x_labels = O.get_xticks()
    O.set_yticklabels([])
    O.set_xticklabels(['%2.1f'%(x/scaling_x) for x in x_labels])
    O.set_xlabel("Amount\n%s"%scaling_label)
    display(g)
    
dropdown_list.observe(read_vals)
# dropdown_list.on_widget_constructed(read_vals)

preview_button = wd.Button(description='Preview Donor')
preview_button.on_click(preview_dists)

calculate_button = wd.Button(description='Show Result')
calculate_button.on_click(show_result)
confidence_label = wd.Label("Confidence")
# min_donation.observe(update_vals, 'value')
# max_donation.observe(update_vals, 'value')
# pessimism_amount.observe(update_vals, 'value')
# optimism_amount.observe(update_vals, 'value')
# pessimism_confidence.observe(update_vals, 'value')
# optimism_confidence.observe(update_vals, 'value')

# HIDDEN
selection_interface = [min_donation, max_donation, optimism_amount, pessimism_amount]
donor_interface = [dropdown_list, confidence_label, optimism_confidence, pessimism_confidence]
button_interface = [preview_button, write_button, calculate_button]
predictions = wd.VBox([ wd.HBox([wd.VBox(donor_interface), wd.VBox(label_list), wd.VBox(selection_interface)]), 
        wd.VBox([ wd.HBox(button_interface), plot_out, results_out ]) ])

In [6]:
predictions